In [1]:
import os
from bs4 import BeautifulSoup
import time
import re
import traceback
from selenium import webdriver
import datetime
from openpyxl import load_workbook, Workbook
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Write Headline and create a new excel sheet
def xl_sheet_headlines(sheet_name):
    wb = Workbook()
    ws = wb.active
    headlines = headlines = ["Case Number", "Assignment Date", "Case Status", "Case Title", "Case Type", "Court", "Active", "Created Time Stamp", "Offense", "Citation Number", "Offense Statue", "Citation Degree", "Offense Date", "defendant Party Name", "defendant address", "defendant City", "defendant State", "defendant Zip", "defendant Gender", "defendant Race", "defendant Date Of Birth", "defendant Ethnicity", "State Party Name", "complainant Party Name", "complainant address", "complainant City", "complainant State", "complainant Zip", "complainant Register Sex offender=", "complainant Interpreter Needed", "complainant Party in Jail", "complainant Marital Status", "Hearing Date Time", "Hearing type", "Hearing Cancelled", "Hearing Created", "Hearing Court Session Name", "Actual Hearing Type", "Hearing Result", "Hearing Location", "Judgment Events", "Citee Name", "Vehicle State", "Vehicle Type", "Citation Fine", "Citation County Location", "Citation Officer Name", "Citation Witness", "Financial Transaction Date", "Total Financial Assessment", "Total Payments", "Balance", "Event Type", "Event Date", "Event Created", "Event Comment", "Report Agency", "Reporting Agency Code", "charges Office Name", "charges Stage", "charges Fine", "charges Degree", "charges Statue", "charges Description", "Bonds", "URL"]
    ws.append(headlines)
    wb.save(sheet_name)

In [4]:
# Write Data On existing sheet
def xl_write(data_write, sheet_name):
    wb = load_workbook(sheet_name)
    work_sheet = wb.active # Get active sheet
    work_sheet.append(data_write)
    wb.save(sheet_name)

In [5]:
def driver_define():
    print('Chromedriver Installing')
    driver_path = ChromeDriverManager().install()
    
    print('Chrome Browser Opening')
    options = Options()
    options.add_experimental_option('excludeSwitches', ['enable-logging'])
    s = Service(driver_path)
    driver = webdriver.Chrome(service=s, options =options)
    driver.maximize_window()
    return driver

In [6]:
def set_login(driver, email, password):
    print('Login on Website...')
    login_page = 'https://prdaws.nccourts.org/rpa_ui/index.html'

    driver.get(login_page)
    time.sleep(0.2)

    ok_button = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//button[@class="btn btn-primary" and text()="OK"]')))
    ok_button.click()
    time.sleep(0.2)

    case_search = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@class="button submit-button" and text()="Case Search"]')))
    case_search.click()
    time.sleep(2)

    # Switch to Last Windows
    driver.switch_to.window(driver.window_handles[-1])

    email_submit = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[type="email"]')))
    email_submit.send_keys(email)
    time.sleep(1)

    # next page
    next_page = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[value="Next"]')))
    next_page.click()
    time.sleep(2)

    password_submit = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[type="password"]')))
    time.sleep(1)
    password_submit.send_keys(password)
    time.sleep(1)

    # Sign In
    sign_in = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-report-event="Signin_Submit"]')))
    sign_in.click()
    time.sleep(1)

    input("Please Completed The 2FA and press Enter")

    # Switch to Last Windows
    driver.switch_to.window(driver.window_handles[0])
    print('Login Completed [OK]')

In [7]:
def set_search_filter(driver, select_county_value, select_case_type_value, select_party_connection_types_value, CS_start_date, CS_end_date):

    print('Set Search Filter')
    search_page_url = 'https://prdaws.nccourts.org/rpa_ui/cases'
    driver.get(search_page_url)
    time.sleep(1)

    print('Go On Advanced Search')
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[text()="Advanced Search"]'))).click()
    time.sleep(1)

    print("Click datesCheckbox")
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[name="datesCheckbox"]'))).click()
    time.sleep(1)

    print("Select County")
    driver.find_element(By.XPATH, '//*[@id="coordinated-search"]//*[text()="Select Counties"]/..').click()

    select_all = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f'//*[@class="dropdown-list"and not(@hidden)]//input[@aria-label="{select_county_value}"]/..')))
    time.sleep(0.4)
    select_all.click()
    print(f">> {select_county_value}")

    print("Case Type")
    driver.find_element(By.XPATH, '//*[@id="coordinated-search"]//*[text()="Select Types"]/..').click()

    infraction_select = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f'//*[@class="dropdown-list"and not(@hidden)]//input[@aria-label="{select_case_type_value}"]/..')))
    time.sleep(0.4)
    infraction_select.click()
    print(f">> {select_county_value}")

    print("Party Connection Types")
    driver.find_element(By.XPATH, '//*[@id="coordinated-search"]//*[text()="Select Conn types"]/..').click()

    defendant_select = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, f'//*[@class="dropdown-list"and not(@hidden)]//input[@aria-label="{select_party_connection_types_value}"]/..')))
    time.sleep(0.4)
    defendant_select.click()
    print(f">> {select_party_connection_types_value}")

    print("Case Filed Start Date")
    start_date_submit = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#case-start-date-input')))
    time.sleep(0.4)
    start_date_submit.send_keys(CS_start_date)
    print(f">> {CS_start_date}")

    print("Case Filed End Date")
    end_date_submit = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#case-end-date-input')))
    time.sleep(0.4)
    end_date_submit.send_keys(CS_end_date)
    print(f">> {CS_end_date}")

    print('Search')
    driver.find_element(By.XPATH, '//*[@id="coordinated-search"]//*[@type="submit" and contains(text(), "Search")]').click()

    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '//strong[text()="Search Result"]')))
    print("Sucessfully Set The Search Criteria [OK]")

In [8]:
def get_view_urls(driver):

    print("Gathring all Cases View Details Urls")
    view_urls = []

    while True:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[class="float-end link-in-title-bar"]')))
        time.sleep(1)
        urls_eles = driver.find_elements(By.CSS_SELECTOR, '[class="float-end link-in-title-bar"]')
        for element in urls_eles:
            view_url = element.get_attribute("href")
            view_urls.append(view_url)

            # element delete
            driver.execute_script("""var element = arguments[0];element.parentNode.removeChild(element);""", element)

        # Visit Next page and if page not exist then break
        has_next_page = driver.find_elements(By.XPATH, '//a[@href="#" and contains(text(), "Next Page") and not(@hidden)]')
        if len(has_next_page) != 0:
            print('Next Page> ', end='')
            next_page = has_next_page[0]
            next_page.click()
            # Wait Until next page load
            WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="coordinated-search"]//*[@type="submit" and contains(text(), "Search") and @disabled]')))
            time.sleep(1)

            # next page load completed
            WebDriverWait(driver, 50).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="coordinated-search"]//*[@type="submit" and contains(text(), "Search")]')))
        else:
            print('\nExtracting Urls From all pages has been completed')
            break
    
    # Fix Duplicate
    view_urls = list(dict.fromkeys(view_urls))
    return view_urls

In [9]:
def extract_single_page(driver, view_url, sheet_name):

    driver.get(view_url)

    # Wait Until Data appears on Page
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '//td[text()="Case Number:"]/following-sibling::td[string-length(text()) > 0] ')))

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # with open('html_data.html') as file:
    #     soup = BeautifulSoup(file.read(), 'html.parser')

    # Emtry String IF data not found
    Case_Number = ""
    Case_Title = ""
    Case_Type = ""
    Court = ""
    Case_Status = ""
    Assignment_Date = ""
    Active = ""
    Created_Time_Stamp = ""
    Offense = ""
    Citation_Number = ""
    Offense_Statue = ""
    Citation_Degree = ""
    Offense_Date = ""
    defendant_soup = ""
    defendant_Party_Name = ""
    defendant_address = ""
    defendant_City = ""
    defendant_State = ""
    defendant_Zip = ""
    defendant_Gender = ""
    defendant_Race = ""
    defendant_Date_Of_Birth = ""
    defendant_Ethnicity = ""
    State_Party_Name = ""
    complainant_soup = ""
    complainant_Party_Name = ""
    complainant_address = ""
    complainant_City = ""
    complainant_State = ""
    complainant_Zip = ""
    complainant_Register_Sex_offender= ""
    complainant_Interpreter_Needed = ""
    complainant_Party_in_Jail = ""
    complainant_Marital_Status = ""
    Hearing_Date_Time = ""
    Hearing_type = ""
    Hearing_Cancelled = ""
    Hearing_Created = ""
    Hearing_Court_Session_Name = ""
    Actual_Hearing_Type = ""
    Hearing_Result = ""
    Hearing_Location = ""
    Judgment_Events = ""
    Citee_Name = ""
    Vehicle_State = ""
    Vehicle_Type = ""
    Citation_Fine = ""
    Citation_County_Location = ""
    Citation_Officer_Name = ""
    Citation_Witness = ""
    Financial_Transaction_Date = ""
    Total_Financial_Assessment = ""
    Total_Payments = ""
    Balance = ""
    Event_Type = ""
    Event_Date = ""
    Event_Created = ""
    Event_Comment = ""
    Report_Agency = ""
    Reporting_Agency_Code = ""
    charges_Office_Name = ""
    charges_Stage = ""
    charges_Fine = ""
    charges_Degree = ""
    charges_Statue = ""
    charges_Description = ""
    Bonds = ""

    # Case Information Summary
    try:
        Case_Number = soup.find('td', string="Case Number:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Case_Title = soup.find('td', string="Case Title:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Case_Type = soup.find('td', string="Case Type:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Court = soup.find('td', string="Court:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Case_Status = soup.find('td', string="Case Status:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Assignment_Date = soup.find('td', string="Assignment Date:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Active = soup.find('td', string="Active:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Created_Time_Stamp = soup.find('td', string="Created Time Stamp:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Offense = soup.find('strong', string="Offense: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Citation_Number = soup.find('strong', string="Citation Number: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Offense_Statue = soup.find('strong', string="Offense Statue: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Citation_Degree = soup.find('strong', string="Citation Degree: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Offense_Date = soup.find('strong', string="Offense Date: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    # Connection Type: Defendant
    try:
        defendant_soup = soup.find('div', string="Connection Type: Defendant").find_previous()
    except:
        #print(traceback.format_exc())
        pass

    try:
        defendant_Party_Name = defendant_soup.find('span', string="Party Name: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        defendant_address = defendant_soup.find('span', string="Address:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        defendant_City = defendant_soup.find('span', string="City: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        defendant_State = defendant_soup.find('span', string="State: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        defendant_Zip = defendant_soup.find('span', string="Zip:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        defendant_Gender = defendant_soup.find('td', string="Gender: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        defendant_Race = defendant_soup.find('td', string="Race: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        defendant_Date_Of_Birth = defendant_soup.find('td', string="Date Of Birth").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        defendant_Ethnicity = defendant_soup.find('td', string="Ethnicity: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    # Connection Type: State
    try:
        State_Party_Name = soup.find('div', string="Connection Type: State").find_previous().find('span', string="Party Name: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    # Connection Type: Complainant
    try:
        complainant_soup = soup.find('div', string="Connection Type: Complainant").find_previous()
    except:
        #print(traceback.format_exc())
        pass

    try:
        complainant_Party_Name = complainant_soup.find('span', string="Party Name: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        complainant_address = complainant_soup.find('span', string="Address:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        complainant_City = complainant_soup.find('span', string="City: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        complainant_State = complainant_soup.find('span', string="State: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        complainant_Zip = complainant_soup.find('span', string="Zip:").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        complainant_Register_Sex_offender= complainant_soup.find('td', string="Register Sex offender: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        complainant_Interpreter_Needed = complainant_soup.find('td', string="Interpreter Needed: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        complainant_Party_in_Jail = complainant_soup.find('td', string="Party in Jail: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        complainant_Marital_Status = complainant_soup.find('td', string="Marital Status: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    # Case Hearings
    try:
        Hearing_Date_Time = soup.select_one('[id="case-hearings"]').find("strong", string="Hearing Date-Time: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Hearing_type = soup.select_one('[id="case-hearings"]').find("strong", string="Hearing Type: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Hearing_Cancelled = soup.select_one('[id="case-hearings"]').find("strong", string="Cancelled: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Hearing_Created = soup.select_one('[id="case-hearings"]').find("strong", string="Created: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Hearing_Court_Session_Name = soup.select_one('[id="case-hearings"]').find("strong", string="Court Session Name: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Actual_Hearing_Type = soup.select_one('[id="case-hearings"]').find("strong", string="Actual Hearing Type: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Hearing_Result = soup.select_one('[id="case-hearings"]').find("strong", string="Hearing Result: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Hearing_Location = soup.select_one('[id="case-hearings"]').find(string=re.compile(r"Hearing Location - Location:.*?")).text.strip()
    except:
        #print(traceback.format_exc())
        pass

    # Judgment Events
    try:
        Judgment_Events = soup.select_one('[id="case-judgments"]').text.split('Judgment Events', 1)[-1].split('Hide', 1)[-1].strip()
    except:
        #print(traceback.format_exc())
        pass

    # Citation
    try:
        Citee_Name = soup.select_one('[id="case-citation"]').find("strong", string="Citee Name: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Vehicle_State = soup.select_one('[id="case-citation"]').find("strong", string="Vehicle State: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Vehicle_Type = soup.select_one('[id="case-citation"]').find("strong", string="Vehicle Type: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Citation_Fine = soup.select_one('[id="case-citation"]').find("strong", string="Fine: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Citation_County_Location = soup.select_one('[id="case-citation"]').find("strong", string="County Location: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Citation_Officer_Name = soup.select_one('[id="case-citation"]').find("strong", string="Officer Name: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Citation_Witness = soup.select_one('[id="case-citation"]').find("strong", string="Witness: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    # Financial
    try:
        Financial_Transaction_Date = soup.select_one('[id="financial-info"]').find(string=re.compile(r"Transaction Date: .*?")).text.split('Transaction Date: ', 1)[-1].strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Total_Financial_Assessment = soup.select_one('[id="financial-info"]').find(string="Total Financial Assessment:").find_next().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Total_Payments = soup.select_one('[id="financial-info"]').find(string=" Total Payments: ").find_next().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Balance = soup.select_one('[id="financial-info"]').find(string=" Balance: ").find_next().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    # Case Events
    try:
        Event_Type = soup.select_one('[id="case-events"]').find("strong", string=re.compile(r"Event Type: .*")).find_previous().text.replace("Event Type:", '').strip().strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Event_Date = soup.select_one('[id="case-events"]').find("strong", string=re.compile(r"Event Date: .*")).find_previous().text.replace("Event Date:", '').strip().strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Event_Created = soup.select_one('[id="case-events"]').find("strong", string=re.compile(r"Created: .*")).find_previous().text.replace("Created:", '').strip().strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Event_Comment = soup.select_one('[id="case-events"]').find("strong", string=re.compile(r"Comment: .*")).find_previous().text.replace("Comment:", '').strip().strip()
    except:
        #print(traceback.format_exc())
        pass

    # Charges
    try:
        Report_Agency = soup.select_one('[id="case-charges"]').find("strong", string="Report Agency: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        Reporting_Agency_Code = soup.select_one('[id="case-charges"]').find("strong", string="Reporting Agency Code: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        charges_Office_Name = soup.select_one('[id="case-charges"]').find("strong", string="Office Name: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        charges_Stage = soup.select_one('[id="case-charges"]').find("strong", string="Stage: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        charges_Fine = soup.select_one('[id="case-charges"]').find("strong", string="Fine: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        charges_Degree = soup.select_one('[id="case-charges"]').find("strong", string="Degree: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        charges_Statue = soup.select_one('[id="case-charges"]').find("strong", string="Statue: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    try:
        charges_Description = soup.select_one('[id="case-charges"]').find("strong", string="Description: ").find_next_sibling().text.strip()
    except:
        #print(traceback.format_exc())
        pass

    # Bonds
    try:
        Bonds = soup.select_one('[id="case-bonds"]').text.split('bonds', 1)[-1].split('Hide', 1)[-1].strip()
    except:
        #print(traceback.format_exc())
        pass

    # Case_Number, Assignment_Date, Case_Status
    write_data = [Case_Number, Assignment_Date, Case_Status, Case_Title, Case_Type, Court, Active, Created_Time_Stamp, Offense, Citation_Number, Offense_Statue, Citation_Degree, Offense_Date, defendant_Party_Name, defendant_address, defendant_City, defendant_State, defendant_Zip, defendant_Gender, defendant_Race, defendant_Date_Of_Birth, defendant_Ethnicity, State_Party_Name, complainant_Party_Name, complainant_address, complainant_City, complainant_State, complainant_Zip, complainant_Register_Sex_offender, complainant_Interpreter_Needed, complainant_Party_in_Jail, complainant_Marital_Status, Hearing_Date_Time, Hearing_type, Hearing_Cancelled, Hearing_Created, Hearing_Court_Session_Name, Actual_Hearing_Type, Hearing_Result, Hearing_Location, Judgment_Events, Citee_Name, Vehicle_State, Vehicle_Type, Citation_Fine, Citation_County_Location, Citation_Officer_Name, Citation_Witness, Financial_Transaction_Date, Total_Financial_Assessment, Total_Payments, Balance, Event_Type, Event_Date, Event_Created, Event_Comment, Report_Agency, Reporting_Agency_Code, charges_Office_Name, charges_Stage, charges_Fine, charges_Degree, charges_Statue, charges_Description, Bonds, view_url]

    # Write on Excel Sheet
    xl_write(write_data, sheet_name)

In [10]:
email = ''
password = ''

In [11]:
# Input Date as MM/DD/YYYY Format
CS_start_date = "10/01/2023"
CS_end_date = "10/05/2023"

In [12]:
select_county_value = "multiselect-select-all"
select_case_type_value = "Infraction"
select_party_connection_types_value = "Defendant"

In [13]:
driver = driver_define()

In [14]:
set_login(driver, email, password)

In [ ]:
try: set_search_filter(driver, select_county_value, select_case_type_value, select_party_connection_types_value, CS_start_date, CS_end_date)
except: set_search_filter(driver, select_county_value, select_case_type_value, select_party_connection_types_value, CS_start_date, CS_end_date)

In [ ]:
view_urls = get_view_urls(driver)

In [ ]:
print('Creating New Excel Sheet')

# create filename for this run. Unique date and time for each file.
scrapeDateTime = datetime.datetime.now().strftime("%B_%d_%Y_%H%M")
#csv_filename = 'Vrbo_Data_Manasota_Key_FL_' + scrapeDateTime + '.csv'
sheet_name = 'nccourts_Data_' + scrapeDateTime + '.xlsx'
# Write Headline
xl_sheet_headlines(sheet_name)

In [ ]:
for view_url in view_urls:
    try:
        extract_single_page(driver, view_url, sheet_name)
    except:
        print(traceback.format_exc())

In [ ]:
driver.quit()